# Task 3: Model Explainability
## Advanced Fraud Detection System

This notebook covers model explainability using SHAP and LIME for understanding fraud detection models.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

# Model explainability libraries
import shap
import lime
from lime.lime_tabular import LimeTabularExplainer

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Initialize SHAP
shap.initjs()

## 1. Load Data and Models

In [ ]:
# Load processed datasets
print("Loading datasets and models...")

try:
    # Load processed data
    fraud_data = pd.read_csv('../data/fraud_data_processed.csv')
    credit_data = pd.read_csv('../data/credit_card_processed.csv')
    print(f"Fraud data shape: {fraud_data.shape}")
    print(f"Credit data shape: {credit_data.shape}")
except FileNotFoundError:
    print("Processed data not found. Loading original data...")
    fraud_data = pd.read_csv('../data/Fraud_Data.csv')
    credit_data = pd.read_csv('../data/creditcard.csv')
    
    # Basic preprocessing for original data
    from sklearn.preprocessing import LabelEncoder
    
    # Process fraud data
    fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
    fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])
    fraud_data['hour_of_day'] = fraud_data['purchase_time'].dt.hour
    fraud_data['day_of_week'] = fraud_data['purchase_time'].dt.dayofweek
    
    le = LabelEncoder()
    for col in ['source', 'browser', 'sex']:
        fraud_data[f'{col}_encoded'] = le.fit_transform(fraud_data[col])
    
    # Select relevant features
    fraud_features = ['purchase_value', 'age', 'hour_of_day', 'day_of_week', 
                     'source_encoded', 'browser_encoded', 'sex_encoded']
    fraud_data = fraud_data[fraud_features + ['class']]

print("Data loaded successfully!")

In [ ]:
# Prepare data for explainability
print("=== DATA PREPARATION ===")

# Fraud data preparation
if 'class' in fraud_data.columns:
    X_fraud = fraud_data.drop(columns=['class'])
    y_fraud = fraud_data['class']
else:
    print("Error: 'class' column not found in fraud data")

# Credit data preparation
if 'Class' in credit_data.columns:
    X_credit = credit_data.drop(columns=['Class'])
    y_credit = credit_data['Class']
else:
    print("Error: 'Class' column not found in credit data")

# Train-test split
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

X_credit_train, X_credit_test, y_credit_train, y_credit_test = train_test_split(
    X_credit, y_credit, test_size=0.2, random_state=42, stratify=y_credit
)

# Feature scaling
scaler_fraud = StandardScaler()
scaler_credit = StandardScaler()

X_fraud_train_scaled = scaler_fraud.fit_transform(X_fraud_train)
X_fraud_test_scaled = scaler_fraud.transform(X_fraud_test)

X_credit_train_scaled = scaler_credit.fit_transform(X_credit_train)
X_credit_test_scaled = scaler_credit.transform(X_credit_test)

print(f"Fraud training set: {X_fraud_train_scaled.shape}")
print(f"Credit training set: {X_credit_train_scaled.shape}")

In [ ]:
# Train models for explainability (if not already available)
print("=== MODEL TRAINING FOR EXPLAINABILITY ===")

# Try to load pre-trained models
try:
    fraud_model = joblib.load('../models/best_fraud_model.pkl')
    credit_model = joblib.load('../models/best_credit_model.pkl')
    print("Loaded pre-trained models successfully!")
except FileNotFoundError:
    print("Pre-trained models not found. Training new models...")
    
    # Train Random Forest models for explainability
    fraud_model = RandomForestClassifier(n_estimators=100, random_state=42)
    credit_model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    fraud_model.fit(X_fraud_train_scaled, y_fraud_train)
    credit_model.fit(X_credit_train_scaled, y_credit_train)
    
    # Evaluate models
    fraud_pred = fraud_model.predict(X_fraud_test_scaled)
    credit_pred = credit_model.predict(X_credit_test_scaled)
    
    print(f"Fraud model accuracy: {accuracy_score(y_fraud_test, fraud_pred):.4f}")
    print(f"Credit model accuracy: {accuracy_score(y_credit_test, credit_pred):.4f}")

# Also train Logistic Regression for comparison
fraud_lr = LogisticRegression(random_state=42, max_iter=1000)
credit_lr = LogisticRegression(random_state=42, max_iter=1000)

fraud_lr.fit(X_fraud_train_scaled, y_fraud_train)
credit_lr.fit(X_credit_train_scaled, y_credit_train)

print("Models prepared for explainability analysis!")

## 2. SHAP Explainability Analysis

In [ ]:
# SHAP Analysis for Fraud Detection Model
print("=== SHAP ANALYSIS - FRAUD MODEL ===")

# Create SHAP explainer for Random Forest
fraud_explainer = shap.TreeExplainer(fraud_model)

# Calculate SHAP values for test set (sample for performance)
sample_size = min(1000, len(X_fraud_test_scaled))
sample_indices = np.random.choice(len(X_fraud_test_scaled), sample_size, replace=False)
X_fraud_sample = X_fraud_test_scaled[sample_indices]
y_fraud_sample = y_fraud_test.iloc[sample_indices]

fraud_shap_values = fraud_explainer.shap_values(X_fraud_sample)

print(f"SHAP values calculated for {sample_size} fraud samples")
print(f"SHAP values shape: {fraud_shap_values[1].shape}")

In [ ]:
# SHAP Summary Plot for Fraud Model
plt.figure(figsize=(12, 8))
shap.summary_plot(fraud_shap_values[1], X_fraud_sample, feature_names=X_fraud.columns, show=False)
plt.title('SHAP Summary Plot - Fraud Detection Model')
plt.tight_layout()
plt.show()

print("SHAP Summary Plot shows:")
print("- Feature importance (x-axis)")
print("- Feature values (color: red=high, blue=low)")
print("- Impact on model output (positive/negative SHAP values)")

In [ ]:
# SHAP Feature Importance Plot
plt.figure(figsize=(10, 6))
shap.summary_plot(fraud_shap_values[1], X_fraud_sample, feature_names=X_fraud.columns, 
                  plot_type="bar", show=False)
plt.title('SHAP Feature Importance - Fraud Detection Model')
plt.tight_layout()
plt.show()

# Calculate mean absolute SHAP values for feature importance
fraud_feature_importance = np.abs(fraud_shap_values[1]).mean(axis=0)
fraud_importance_df = pd.DataFrame({
    'feature': X_fraud.columns,
    'importance': fraud_feature_importance
}).sort_values('importance', ascending=False)

print("\nTop 5 Most Important Features (Fraud Model):")
print(fraud_importance_df.head())

In [ ]:
# SHAP Dependence Plots for top features
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

top_features = fraud_importance_df.head(4)['feature'].tolist()

for i, feature in enumerate(top_features):
    feature_idx = list(X_fraud.columns).index(feature)
    plt.sca(axes[i])
    shap.dependence_plot(feature_idx, fraud_shap_values[1], X_fraud_sample, 
                        feature_names=X_fraud.columns, show=False)
    plt.title(f'SHAP Dependence Plot - {feature}')

plt.tight_layout()
plt.show()

print("Dependence plots show how feature values affect SHAP values (model output)")

In [ ]:
# SHAP Analysis for Credit Card Model
print("=== SHAP ANALYSIS - CREDIT CARD MODEL ===")

# Create SHAP explainer for Credit Card model
credit_explainer = shap.TreeExplainer(credit_model)

# Calculate SHAP values for test set (sample for performance)
sample_size = min(1000, len(X_credit_test_scaled))
sample_indices = np.random.choice(len(X_credit_test_scaled), sample_size, replace=False)
X_credit_sample = X_credit_test_scaled[sample_indices]
y_credit_sample = y_credit_test.iloc[sample_indices]

credit_shap_values = credit_explainer.shap_values(X_credit_sample)

print(f"SHAP values calculated for {sample_size} credit samples")
print(f"SHAP values shape: {credit_shap_values[1].shape}")

In [ ]:
# SHAP Summary Plot for Credit Card Model
plt.figure(figsize=(12, 10))
# Select top 15 features for better visualization
credit_feature_importance = np.abs(credit_shap_values[1]).mean(axis=0)
top_15_indices = np.argsort(credit_feature_importance)[-15:]

shap.summary_plot(credit_shap_values[1][:, top_15_indices], 
                  X_credit_sample[:, top_15_indices], 
                  feature_names=[X_credit.columns[i] for i in top_15_indices], 
                  show=False)
plt.title('SHAP Summary Plot - Credit Card Fraud Model (Top 15 Features)')
plt.tight_layout()
plt.show()

In [ ]:
# SHAP Feature Importance for Credit Model
plt.figure(figsize=(10, 8))
shap.summary_plot(credit_shap_values[1][:, top_15_indices], 
                  X_credit_sample[:, top_15_indices], 
                  feature_names=[X_credit.columns[i] for i in top_15_indices],
                  plot_type="bar", show=False)
plt.title('SHAP Feature Importance - Credit Card Fraud Model (Top 15 Features)')
plt.tight_layout()
plt.show()

# Create importance dataframe
credit_importance_df = pd.DataFrame({
    'feature': X_credit.columns,
    'importance': credit_feature_importance
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features (Credit Card Model):")
print(credit_importance_df.head(10))

In [ ]:
# SHAP Force Plot for Individual Predictions
print("=== SHAP FORCE PLOTS FOR INDIVIDUAL PREDICTIONS ===")

# Find fraud and non-fraud examples
fraud_indices = np.where(y_fraud_sample == 1)[0]
non_fraud_indices = np.where(y_fraud_sample == 0)[0]

if len(fraud_indices) > 0 and len(non_fraud_indices) > 0:
    # Force plot for a fraud case
    fraud_idx = fraud_indices[0]
    print(f"\nForce plot for FRAUD case (index {fraud_idx}):")
    
    plt.figure(figsize=(12, 4))
    shap.force_plot(fraud_explainer.expected_value[1], 
                   fraud_shap_values[1][fraud_idx], 
                   X_fraud_sample[fraud_idx], 
                   feature_names=X_fraud.columns,
                   matplotlib=True, show=False)
    plt.title('SHAP Force Plot - Fraud Case')
    plt.tight_layout()
    plt.show()
    
    # Force plot for a non-fraud case
    non_fraud_idx = non_fraud_indices[0]
    print(f"\nForce plot for NON-FRAUD case (index {non_fraud_idx}):")
    
    plt.figure(figsize=(12, 4))
    shap.force_plot(fraud_explainer.expected_value[1], 
                   fraud_shap_values[1][non_fraud_idx], 
                   X_fraud_sample[non_fraud_idx], 
                   feature_names=X_fraud.columns,
                   matplotlib=True, show=False)
    plt.title('SHAP Force Plot - Non-Fraud Case')
    plt.tight_layout()
    plt.show()
    
    print("Force plots show:")
    print("- Base value (expected model output)")
    print("- Features pushing prediction higher (red) or lower (blue)")
    print("- Final prediction value")
else:
    print("Not enough fraud/non-fraud cases in sample for force plots")

## 3. LIME Explainability Analysis

In [ ]:
# LIME Analysis for Fraud Detection Model
print("=== LIME ANALYSIS - FRAUD MODEL ===")

# Create LIME explainer
fraud_lime_explainer = LimeTabularExplainer(
    X_fraud_train_scaled,
    feature_names=X_fraud.columns,
    class_names=['Non-Fraud', 'Fraud'],
    mode='classification',
    discretize_continuous=True
)

print("LIME explainer created for fraud model")

In [ ]:
# LIME explanation for individual predictions
def explain_with_lime(explainer, model, instance, instance_idx, title):
    """Generate LIME explanation for a single instance"""
    explanation = explainer.explain_instance(
        instance, 
        model.predict_proba, 
        num_features=10
    )
    
    # Plot explanation
    fig = explanation.as_pyplot_figure()
    fig.suptitle(f'LIME Explanation - {title} (Instance {instance_idx})', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Print explanation
    print(f"\n{title} - Top feature contributions:")
    for feature, weight in explanation.as_list():
        print(f"  {feature}: {weight:.4f}")
    
    return explanation

# Explain fraud cases
if len(fraud_indices) > 0:
    fraud_instance_idx = fraud_indices[0]
    fraud_instance = X_fraud_sample[fraud_instance_idx]
    
    fraud_explanation = explain_with_lime(
        fraud_lime_explainer, fraud_model, fraud_instance, 
        fraud_instance_idx, "Fraud Case"
    )

# Explain non-fraud cases
if len(non_fraud_indices) > 0:
    non_fraud_instance_idx = non_fraud_indices[0]
    non_fraud_instance = X_fraud_sample[non_fraud_instance_idx]
    
    non_fraud_explanation = explain_with_lime(
        fraud_lime_explainer, fraud_model, non_fraud_instance, 
        non_fraud_instance_idx, "Non-Fraud Case"
    )

In [ ]:
# LIME Analysis for Credit Card Model
print("=== LIME ANALYSIS - CREDIT CARD MODEL ===")

# Create LIME explainer for credit card model
credit_lime_explainer = LimeTabularExplainer(
    X_credit_train_scaled,
    feature_names=X_credit.columns,
    class_names=['Non-Fraud', 'Fraud'],
    mode='classification',
    discretize_continuous=True
)

# Find fraud and non-fraud examples in credit data
credit_fraud_indices = np.where(y_credit_sample == 1)[0]
credit_non_fraud_indices = np.where(y_credit_sample == 0)[0]

# Explain credit card fraud cases
if len(credit_fraud_indices) > 0:
    credit_fraud_idx = credit_fraud_indices[0]
    credit_fraud_instance = X_credit_sample[credit_fraud_idx]
    
    credit_fraud_explanation = explain_with_lime(
        credit_lime_explainer, credit_model, credit_fraud_instance, 
        credit_fraud_idx, "Credit Card Fraud Case"
    )

# Explain credit card non-fraud cases
if len(credit_non_fraud_indices) > 0:
    credit_non_fraud_idx = credit_non_fraud_indices[0]
    credit_non_fraud_instance = X_credit_sample[credit_non_fraud_idx]
    
    credit_non_fraud_explanation = explain_with_lime(
        credit_lime_explainer, credit_model, credit_non_fraud_instance, 
        credit_non_fraud_idx, "Credit Card Non-Fraud Case"
    )

## 4. Comparison of SHAP vs LIME

In [ ]:
# Compare SHAP and LIME explanations
print("=== SHAP vs LIME COMPARISON ===")

def compare_explanations(shap_values, lime_explanation, feature_names, instance_idx, title):
    """Compare SHAP and LIME explanations for the same instance"""
    
    # Get SHAP values for the instance
    shap_instance = shap_values[instance_idx]
    
    # Get LIME values
    lime_dict = dict(lime_explanation.as_list())
    
    # Create comparison dataframe
    comparison_data = []
    for i, feature in enumerate(feature_names):
        shap_val = shap_instance[i]
        lime_val = 0
        
        # Find corresponding LIME value
        for lime_feature, lime_weight in lime_dict.items():
            if feature in lime_feature or lime_feature in feature:
                lime_val = lime_weight
                break
        
        comparison_data.append({
            'Feature': feature,
            'SHAP': shap_val,
            'LIME': lime_val
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Plot comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # SHAP values
    top_shap = comparison_df.nlargest(10, 'SHAP', keep='all')
    ax1.barh(range(len(top_shap)), top_shap['SHAP'])
    ax1.set_yticks(range(len(top_shap)))
    ax1.set_yticklabels(top_shap['Feature'])
    ax1.set_title(f'SHAP Values - {title}')
    ax1.set_xlabel('SHAP Value')
    
    # LIME values
    lime_data = [(k, v) for k, v in lime_dict.items()]
    lime_features, lime_values = zip(*lime_data) if lime_data else ([], [])
    
    if lime_values:
        ax2.barh(range(len(lime_values)), lime_values)
        ax2.set_yticks(range(len(lime_values)))
        ax2.set_yticklabels([f[:20] + '...' if len(f) > 20 else f for f in lime_features])
        ax2.set_title(f'LIME Values - {title}')
        ax2.set_xlabel('LIME Weight')
    
    plt.tight_layout()
    plt.show()
    
    return comparison_df

# Compare explanations for fraud case
if len(fraud_indices) > 0 and 'fraud_explanation' in locals():
    fraud_comparison = compare_explanations(
        fraud_shap_values[1], fraud_explanation, X_fraud.columns, 
        fraud_indices[0], "Fraud Case"
    )
    print("\nFraud Case - SHAP vs LIME Comparison:")
    print(fraud_comparison.head(10))

## 5. Global vs Local Explanations

In [ ]:
# Global vs Local Explanations Analysis
print("=== GLOBAL vs LOCAL EXPLANATIONS ===")

# Global explanation using SHAP (feature importance across all samples)
global_importance = np.abs(fraud_shap_values[1]).mean(axis=0)
global_df = pd.DataFrame({
    'Feature': X_fraud.columns,
    'Global_Importance': global_importance
}).sort_values('Global_Importance', ascending=False)

print("\nGlobal Feature Importance (SHAP):")
print(global_df.head(10))

# Local explanations for multiple instances
print("\nLocal Explanations for Different Instances:")

# Analyze local explanations for different types of cases
local_explanations = []
instance_types = []

# Sample different instances
sample_indices_analysis = np.random.choice(len(X_fraud_sample), min(5, len(X_fraud_sample)), replace=False)

for idx in sample_indices_analysis:
    instance_shap = fraud_shap_values[1][idx]
    instance_type = "Fraud" if y_fraud_sample.iloc[idx] == 1 else "Non-Fraud"
    
    # Get top 3 features for this instance
    top_indices = np.argsort(np.abs(instance_shap))[-3:]
    top_features = [X_fraud.columns[i] for i in top_indices]
    top_values = [instance_shap[i] for i in top_indices]
    
    local_explanations.append({
        'Instance': idx,
        'Type': instance_type,
        'Top_Features': top_features,
        'SHAP_Values': top_values
    })
    
    print(f"\nInstance {idx} ({instance_type}):")
    for feature, value in zip(top_features, top_values):
        print(f"  {feature}: {value:.4f}")

# Visualize global vs local importance
plt.figure(figsize=(12, 8))

# Global importance
plt.subplot(2, 1, 1)
top_global = global_df.head(10)
plt.barh(range(len(top_global)), top_global['Global_Importance'])
plt.yticks(range(len(top_global)), top_global['Feature'])
plt.title('Global Feature Importance (Average |SHAP| values)')
plt.xlabel('Average Absolute SHAP Value')

# Local importance variation
plt.subplot(2, 1, 2)
feature_std = np.std(np.abs(fraud_shap_values[1]), axis=0)
std_df = pd.DataFrame({
    'Feature': X_fraud.columns,
    'Importance_Std': feature_std
}).sort_values('Importance_Std', ascending=False)

top_std = std_df.head(10)
plt.barh(range(len(top_std)), top_std['Importance_Std'])
plt.yticks(range(len(top_std)), top_std['Feature'])
plt.title('Feature Importance Variation Across Instances (Std of |SHAP| values)')
plt.xlabel('Standard Deviation of Absolute SHAP Values')

plt.tight_layout()
plt.show()

print("\nInsights:")
print("- Global importance shows which features are generally most important")
print("- High variation indicates features that are important for some instances but not others")
print("- Local explanations help understand individual predictions")

## 6. Model Comparison Using Explainability

In [ ]:
# Compare different models using explainability
print("=== MODEL COMPARISON USING EXPLAINABILITY ===")

# SHAP analysis for Logistic Regression
fraud_lr_explainer = shap.LinearExplainer(fraud_lr, X_fraud_train_scaled)
fraud_lr_shap_values = fraud_lr_explainer.shap_values(X_fraud_sample)

# Compare feature importance between Random Forest and Logistic Regression
rf_importance = np.abs(fraud_shap_values[1]).mean(axis=0)
lr_importance = np.abs(fraud_lr_shap_values).mean(axis=0)

comparison_models_df = pd.DataFrame({
    'Feature': X_fraud.columns,
    'Random_Forest': rf_importance,
    'Logistic_Regression': lr_importance
})

# Plot comparison
plt.figure(figsize=(12, 8))

# Scatter plot of feature importance
plt.subplot(2, 1, 1)
plt.scatter(comparison_models_df['Random_Forest'], comparison_models_df['Logistic_Regression'])
plt.xlabel('Random Forest Importance')
plt.ylabel('Logistic Regression Importance')
plt.title('Feature Importance Comparison: Random Forest vs Logistic Regression')

# Add feature labels for top features
for i, feature in enumerate(comparison_models_df['Feature']):
    if (comparison_models_df.iloc[i]['Random_Forest'] > 0.01 or 
        comparison_models_df.iloc[i]['Logistic_Regression'] > 0.01):
        plt.annotate(feature, 
                    (comparison_models_df.iloc[i]['Random_Forest'], 
                     comparison_models_df.iloc[i]['Logistic_Regression']),
                    fontsize=8, alpha=0.7)

# Add diagonal line
max_val = max(comparison_models_df['Random_Forest'].max(), 
              comparison_models_df['Logistic_Regression'].max())
plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.5)

# Bar plot comparison for top features
plt.subplot(2, 1, 2)
top_features_combined = comparison_models_df.nlargest(8, 'Random_Forest')

x = np.arange(len(top_features_combined))
width = 0.35

plt.bar(x - width/2, top_features_combined['Random_Forest'], width, 
        label='Random Forest', alpha=0.8)
plt.bar(x + width/2, top_features_combined['Logistic_Regression'], width, 
        label='Logistic Regression', alpha=0.8)

plt.xlabel('Features')
plt.ylabel('SHAP Importance')
plt.title('Top Features Comparison: Random Forest vs Logistic Regression')
plt.xticks(x, top_features_combined['Feature'], rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

print("\nTop 5 Features - Random Forest:")
print(comparison_models_df.nlargest(5, 'Random_Forest')[['Feature', 'Random_Forest']])

print("\nTop 5 Features - Logistic Regression:")
print(comparison_models_df.nlargest(5, 'Logistic_Regression')[['Feature', 'Logistic_Regression']])

## 7. Actionable Insights and Recommendations

In [ ]:
# Generate actionable insights
print("=== ACTIONABLE INSIGHTS AND RECOMMENDATIONS ===")

# Analyze feature patterns in fraud vs non-fraud cases
fraud_cases_shap = fraud_shap_values[1][y_fraud_sample == 1]
non_fraud_cases_shap = fraud_shap_values[1][y_fraud_sample == 0]

if len(fraud_cases_shap) > 0 and len(non_fraud_cases_shap) > 0:
    fraud_mean_shap = np.mean(fraud_cases_shap, axis=0)
    non_fraud_mean_shap = np.mean(non_fraud_cases_shap, axis=0)
    
    # Features that distinguish fraud from non-fraud
    difference = fraud_mean_shap - non_fraud_mean_shap
    
    distinguishing_features = pd.DataFrame({
        'Feature': X_fraud.columns,
        'Fraud_SHAP': fraud_mean_shap,
        'Non_Fraud_SHAP': non_fraud_mean_shap,
        'Difference': difference
    }).sort_values('Difference', key=abs, ascending=False)
    
    print("\nFeatures that most distinguish Fraud from Non-Fraud:")
    print(distinguishing_features.head(10))
    
    # Visualize distinguishing features
    plt.figure(figsize=(12, 6))
    top_distinguishing = distinguishing_features.head(8)
    
    x = np.arange(len(top_distinguishing))
    width = 0.35
    
    plt.bar(x - width/2, top_distinguishing['Fraud_SHAP'], width, 
            label='Fraud Cases', alpha=0.8, color='red')
    plt.bar(x + width/2, top_distinguishing['Non_Fraud_SHAP'], width, 
            label='Non-Fraud Cases', alpha=0.8, color='blue')
    
    plt.xlabel('Features')
    plt.ylabel('Average SHAP Value')
    plt.title('Average SHAP Values: Fraud vs Non-Fraud Cases')
    plt.xticks(x, top_distinguishing['Feature'], rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Generate business recommendations
print("\n=== BUSINESS RECOMMENDATIONS ===")
print("\n1. FEATURE MONITORING:")
top_3_features = fraud_importance_df.head(3)['feature'].tolist()
for i, feature in enumerate(top_3_features, 1):
    print(f"   {i}. Monitor '{feature}' closely - highest predictive power for fraud")

print("\n2. RISK SCORING:")
print("   - Implement real-time risk scoring based on top features")
print("   - Set thresholds for automatic flagging of suspicious transactions")

print("\n3. MODEL INTERPRETATION:")
print("   - Use SHAP values for explaining individual fraud predictions to investigators")
print("   - LIME explanations can help in regulatory compliance and audit trails")

print("\n4. FEATURE ENGINEERING:")
high_variation_features = std_df.head(3)['Feature'].tolist()
print(f"   - Features with high variation ({', '.join(high_variation_features)}) may benefit from")
print("     additional context or interaction terms")

print("\n5. MODEL IMPROVEMENT:")
print("   - Random Forest and Logistic Regression show different feature importance patterns")
print("   - Consider ensemble methods to leverage both perspectives")
print("   - Focus data collection efforts on top distinguishing features")

In [ ]:
# Save explainability results
print("=== SAVING EXPLAINABILITY RESULTS ===")

import os
os.makedirs('../explainability_results', exist_ok=True)

# Save feature importance results
fraud_importance_df.to_csv('../explainability_results/fraud_feature_importance_shap.csv', index=False)
credit_importance_df.to_csv('../explainability_results/credit_feature_importance_shap.csv', index=False)

# Save model comparison results
comparison_models_df.to_csv('../explainability_results/model_comparison_shap.csv', index=False)

# Save distinguishing features
if 'distinguishing_features' in locals():
    distinguishing_features.to_csv('../explainability_results/fraud_distinguishing_features.csv', index=False)

# Save SHAP values for future use
np.save('../explainability_results/fraud_shap_values.npy', fraud_shap_values[1])
np.save('../explainability_results/credit_shap_values.npy', credit_shap_values[1])

print("Explainability results saved successfully!")
print("\nFiles saved:")
print("- fraud_feature_importance_shap.csv")
print("- credit_feature_importance_shap.csv")
print("- model_comparison_shap.csv")
print("- fraud_distinguishing_features.csv")
print("- SHAP values arrays (.npy files)")

print("\n=== TASK 3 COMPLETED SUCCESSFULLY ===")
print("All model explainability steps have been completed:")
print("✅ SHAP analysis for both fraud and credit card models")
print("✅ LIME analysis for individual prediction explanations")
print("✅ Global vs local explainability comparison")
print("✅ Model comparison using explainability metrics")
print("✅ Actionable business insights generated")
print("✅ Explainability results saved for deployment")
print("\nReady for Task 4: Model Deployment and API Development!")